# Giới Thiệu

Bạn đã thấy (và sử dụng) các kỹ thuật để trích xuất thông tin tổng quát từ một mô hình học máy. Nhưng nếu bạn muốn phân tích cách mô hình hoạt động cho một dự đoán cụ thể thì sao?

**SHAP Values** (*SHapley Additive exPlanations*) giúp phân tách một dự đoán để cho thấy ảnh hưởng của từng đặc trưng. Khi nào bạn có thể sử dụng SHAP?

- Một mô hình quyết định rằng ngân hàng không nên cho ai đó vay tiền, và ngân hàng có nghĩa vụ pháp lý phải giải thích lý do từ chối khoản vay.
- Một nhà cung cấp dịch vụ y tế muốn xác định các yếu tố rủi ro gây bệnh của từng bệnh nhân để có thể đưa ra các biện pháp can thiệp sức khỏe phù hợp.

Bạn sẽ sử dụng SHAP Values để giải thích các dự đoán cá nhân trong bài học này. Trong bài học tiếp theo, bạn sẽ thấy cách tổng hợp chúng thành những hiểu biết mạnh mẽ ở cấp độ mô hình.

---

# Cách Hoạt Động

SHAP values diễn giải tác động của một giá trị đặc trưng nhất định bằng cách so sánh với dự đoán nếu đặc trưng đó có giá trị cơ sở (*baseline value*).

Một ví dụ sẽ giúp làm rõ điều này. Chúng ta tiếp tục sử dụng bài toán dự đoán trong bóng đá từ các bài học về **[tầm quan trọng hoán vị](https://www.kaggle.com/dansbecker/permutation-importance)** và **[biểu đồ phụ thuộc một phần](https://www.kaggle.com/dansbecker/partial-plots)**.

Trong các bài học trước, chúng ta đã dự đoán liệu một đội có cầu thủ giành danh hiệu *Cầu thủ xuất sắc nhất trận* (*Man of the Match*) hay không.

Chúng ta có thể đặt câu hỏi:
- Dự đoán bị ảnh hưởng bao nhiêu bởi việc đội bóng ghi 3 bàn thắng?

Nhưng câu hỏi này sẽ dễ trả lời hơn dưới dạng số cụ thể nếu được diễn đạt lại như sau:
- Dự đoán bị ảnh hưởng bao nhiêu bởi việc đội ghi 3 bàn thắng, **so với một số bàn thắng cơ sở?**

Dĩ nhiên, mỗi đội có nhiều đặc trưng khác nhau. Vì vậy, nếu chúng ta trả lời câu hỏi cho `số bàn thắng`, chúng ta cũng có thể lặp lại quá trình này cho tất cả các đặc trưng khác.

SHAP values thực hiện điều này theo cách đảm bảo một tính chất quan trọng. Cụ thể, chúng ta có thể phân tách một dự đoán bằng phương trình sau:

```sum(SHAP values cho tất cả đặc trưng) = dự đoán cho đội - dự đoán với giá trị cơ sở```

Tức là, tổng SHAP values của tất cả đặc trưng sẽ giải thích tại sao dự đoán của tôi khác với dự đoán cơ sở. Điều này cho phép chúng ta biểu diễn một dự đoán trong một biểu đồ như sau:

![Imgur](https://storage.googleapis.com/kaggle-media/learn/images/JVD2U7k.png)

*Nếu bạn muốn xem hình lớn hơn, [bấm vào đây](https://storage.googleapis.com/kaggle-media/learn/images/JVD2U7k.png).*


### Cách Diễn Giải SHAP Values

Chúng ta dự đoán giá trị **0.7**, trong khi giá trị cơ sở (*base_value*) là **0.4979**. 

- Các giá trị đặc trưng làm **tăng** dự đoán được hiển thị bằng **màu hồng**, và kích thước trực quan thể hiện độ lớn của tác động.
- Các giá trị đặc trưng làm **giảm** dự đoán được hiển thị bằng **màu xanh lam**.

Tác động lớn nhất đến dự đoán đến từ đặc trưng `Goals Scored = 2`, làm tăng giá trị dự đoán. Trong khi đó, giá trị kiểm soát bóng (*ball possession*) có ảnh hưởng đáng kể trong việc **giảm** dự đoán.

Nếu bạn lấy tổng độ dài của các thanh màu xanh trừ đi tổng độ dài của các thanh màu hồng, bạn sẽ có khoảng cách từ giá trị cơ sở đến đầu ra dự đoán.

Có một số yếu tố phức tạp trong phương pháp này để đảm bảo rằng giá trị cơ sở cộng với tổng ảnh hưởng riêng lẻ của từng đặc trưng bằng đúng giá trị dự đoán. Điều này không đơn giản như nghe có vẻ. Chúng ta sẽ không đi sâu vào chi tiết đó ở đây, vì nó không cần thiết để áp dụng kỹ thuật này. Nếu bạn muốn tìm hiểu lý thuyết chi tiết hơn, có thể tham khảo [bài viết này](https://towardsdatascience.com/one-feature-attribution-method-to-supposedly-rule-them-all-shapley-values-f3e04534983d).

---

# Mã Code Để Tính SHAP Values  

Chúng ta sẽ sử dụng thư viện tuyệt vời [SHAP](https://github.com/slundberg/shap) để tính toán SHAP values.

Trong ví dụ này, chúng ta sẽ sử dụng lại mô hình đã thấy trước đó với dữ liệu bóng đá.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

data = pd.read_csv('FIFA 2018 Statistics.csv')
y = (data['Man of the Match'] == "Yes")  # Convert from string "Yes"/"No" to binary
feature_names = [i for i in data.columns if data[i].dtype in [np.int64, np.int64]]
X = data[feature_names]
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
my_model = RandomForestClassifier(random_state=0).fit(train_X, train_y)

Chúng ta sẽ xem xét SHAP values cho một hàng dữ liệu duy nhất trong tập dữ liệu (ở đây, chúng ta tùy ý chọn hàng số 5). 

Để có cái nhìn tổng quan, trước tiên chúng ta sẽ xem xét dự đoán thô của mô hình trước khi đi vào SHAP values.

In [2]:
row_to_show = 5
data_for_prediction = val_X.iloc[row_to_show]  # use 1 row of data here. Could use multiple rows if desired
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)


my_model.predict_proba(data_for_prediction_array)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


array([[0.29, 0.71]])

Đội bóng này có xác suất **70%** để có một cầu thủ giành danh hiệu "Cầu thủ xuất sắc nhất trận" (*Man of the Match*).

Bây giờ, chúng ta sẽ chuyển sang mã code để tính SHAP values cho dự đoán này.

In [3]:
import shap  # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(my_model)

# Calculate Shap values
shap_values = explainer.shap_values(data_for_prediction)

Đối tượng `shap_values` ở trên là một danh sách chứa hai mảng:

- Mảng đầu tiên chứa SHAP values cho kết quả tiêu cực (đội **không** giành danh hiệu).
- Mảng thứ hai chứa SHAP values cho kết quả tích cực (đội **giành** danh hiệu).

Thông thường, chúng ta quan tâm đến dự đoán cho kết quả tích cực, nên chúng ta sẽ lấy SHAP values cho trường hợp đó (`shap_values[1]`).

Việc xem xét các mảng dữ liệu thô khá rườm rà, nhưng thư viện SHAP cung cấp một cách trực quan để hiển thị kết quả.

In [6]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], data_for_prediction)

NameError: name 'shap' is not defined

Nếu bạn quan sát kỹ đoạn mã nơi chúng ta tạo SHAP values, bạn sẽ thấy rằng chúng ta sử dụng `shap.TreeExplainer(my_model)`. Tuy nhiên, thư viện SHAP có các bộ giải thích (*explainers*) cho mọi loại mô hình:

- `shap.DeepExplainer` hoạt động với các mô hình Học sâu (*Deep Learning*).
- `shap.KernelExplainer` có thể áp dụng cho tất cả các mô hình, nhưng chậm hơn so với các Explainers khác và chỉ cung cấp kết quả gần đúng thay vì giá trị SHAP chính xác.

Dưới đây là một ví dụ sử dụng `KernelExplainer` để tạo ra kết quả tương tự. Kết quả không hoàn toàn giống nhau vì `KernelExplainer` chỉ đưa ra một kết quả xấp xỉ. Tuy nhiên, thông tin rút ra từ kết quả vẫn tương tự nhau.

In [5]:
# use Kernel SHAP to explain test set predictions
k_explainer = shap.KernelExplainer(my_model.predict_proba, train_X)
k_shap_values = k_explainer.shap_values(data_for_prediction)
shap.force_plot(k_explainer.expected_value[1], k_shap_values[1], data_for_prediction)

X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.

# Đến Lượt Bạn

SHAP values thật tuyệt vời! Hãy tận hưởng việc áp dụng chúng, cùng với các công cụ khác mà bạn đã học, để **[giải quyết một bài toán khoa học dữ liệu hoàn chỉnh](https://www.kaggle.com/kernels/fork/1637226)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/machine-learning-explainability/discussion) to chat with other learners.*